<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/sandboxes/BB/analysis/quac_triviaqa_text_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metrics: Text Evaluation

### Imports/Setup

In [23]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 25.3 MB/s 
     |████████████████████████████████| 182 kB 58.2 MB/s 
     |████████████████████████████████| 7.6 MB 54.5 MB/s 


In [24]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 17.1 MB/s 


In [25]:
from os import listdir
from os.path import isfile, join

import csv
import json
import pprint

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_colwidth", None)

In [2]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [19]:
evaluation_root = "/content/drive/MyDrive/w266 NLP Final Project/Evaluation/"
filename = join(evaluation_root, "evaluation_database.json")

# Read JSON into dataframe
evaluation_df = pd.read_json(filename)

### Clean up evaluation dataframe to only include columns needed for text analysis

In [4]:
# Clean up evaluation db df to only include columns needed for text analysis
clean_eval_df = evaluation_df[
    [
        "nickname",
        "base_model",
        "trained_on",
        "tested_on",
        "hyperparameter",
        "target",
        "prediction",
        "bleu",
        "rougeL",
        "meteor",
        "bertscore-f1",
        "use",
    ]
]

clean_eval_df

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
0,bart_nq_nq,bart,nq,nq,{'defaults': True},what was the real name of saudi arabia,what is the name of the kingdom of saudi arabia,0.000000,0.666667,0.623306,0.915830,0.794086
1,bart_nq_nq,bart,nq,nq,{'defaults': True},whats the most liked picture on instagram 2018,what is the most liked picture on instagram,0.680375,0.750000,0.864796,0.932259,0.884909
2,bart_nq_nq,bart,nq,nq,{'defaults': True},where does the movie proof of life take place,where does alice go in the new movie,0.000000,0.470588,0.354635,0.835192,0.517867
3,bart_nq_nq,bart,nq,nq,{'defaults': True},where is net profit on the balance sheet,where does net profit come from in a financial...,0.000000,0.333333,0.311653,0.856130,0.728012
4,bart_nq_nq,bart,nq,nq,{'defaults': True},when was fingerprinting first used by the police,when was fingerprint technology first used in ...,0.000000,0.588235,0.694444,0.876353,0.704206
...,...,...,...,...,...,...,...,...,...,...,...,...
286285,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},Which ex British daily newspaper was first pub...,What is the name of the city of Manchester?,0.000000,0.093023,0.058309,0.679618,0.322748
286286,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},Which cartoon character lived in Bunkerton Cas...,What was the name of Lord Marmaduke of Bunkerton?,0.000000,0.125000,0.121951,0.794428,0.515345
286287,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},In which prison was 'Amy' born in the novel 'L...,What is the name of the prison in Charles Dick...,0.000000,0.416667,0.248227,0.821790,0.593840
286288,T5_amalgam_triviaqa,T5,amalgam,triviaqa,{'defaults': True},The Sign Of Four was a detective story written...,Who wrote the novels 'The Sign of Four' and 'A...,0.000000,0.400000,0.381426,0.796076,0.585255


In [5]:
clean_eval_df.describe()

,bleu,rougeL,meteor,bertscore-f1,use
count,286290.000000,286290.000000,286290.000000,286290.000000,286290.000000
mean,0.048842,0.295583,0.279015,0.789801,0.483113
std,0.158423,0.233494,0.234177,0.082735,0.246386
min,0.000000,0.000000,0.000000,0.521592,-0.147977
25%,0.000000,0.125000,0.097087,0.729552,0.291765
50%,0.000000,0.250000,0.200000,0.784537,0.487792
75%,0.000000,0.428571,0.408791,0.845546,0.671069
max,1.000000,1.000000,0.999937,1.000000,1.000000


### Find the overall averages for each prediction set and metric

In [8]:
# Find the overall averages for each prediction set and metric
eval_means_df = clean_eval_df.groupby(["nickname"]).mean()

# Reset index so able to groupby and sort below to find top scoring prediction sets for each metric
eval_means_df = eval_means_df.reset_index("nickname")

eval_means_df = eval_means_df[["nickname", "bleu", "rougeL", "meteor", "bertscore-f1", "use"]]
eval_means_df

,nickname,bleu,rougeL,meteor,bertscore-f1,use
0,T5_amalgam_nq,0.169746,0.527758,0.497871,0.858036,0.719863
1,T5_amalgam_quac,0.053671,0.249039,0.262385,0.778060,0.378001
2,T5_amalgam_squad,0.142885,0.470772,0.469798,0.867048,0.656399
3,T5_amalgam_triviaqa,0.066390,0.359130,0.337926,0.822766,0.601577
4,T5_nq_nq,0.172979,0.527675,0.497261,0.859476,0.718928
5,T5_nq_quac,0.001471,0.151900,0.111625,0.696885,0.287737
6,T5_nq_squad,0.017446,0.346182,0.273460,0.791693,0.554671
7,T5_nq_triviaqa,0.007450,0.278408,0.189823,0.766649,0.518441
8,T5_quac_nq,0.000000,0.068525,0.040712,0.662784,0.180669
9,T5_quac_quac,0.023456,0.173453,0.189914,0.756046,0.303081


### Create BART and T5 dataframes to focus on

In [11]:
# Create BART dataframe
bart_df = clean_eval_df[clean_eval_df["base_model"] == "bart"]

In [12]:
# Create T5 dataframe
t5_df = clean_eval_df[clean_eval_df["base_model"] == "T5"]

## Model Evaluation

### Examples the models struggle with

Tested on QuAC

In [29]:
# Find predictions with low semantic metric scores when tested on QuAC

quac_low = clean_eval_df[
    (clean_eval_df["trained_on"] == "amalgam")
    & (clean_eval_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (clean_eval_df["use"] <= 0.3)   # bottom 25th percentile use
    & (clean_eval_df["tested_on"] == "quac")
]

quac_low = quac_low.sort_values(by="use", ascending=False)

quac_low

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
119694,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},How long was taking barbiturates?,What did the autopsy show about her stomach?,0.0,0.000000,0.079365,0.707513,0.299898
119134,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},Did he go on to any other notable matches?,Are there any other interesting aspects about this article?,0.0,0.222222,0.255556,0.729418,0.299652
264748,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},Did they win any awards for these singles,How did the song do on the charts?,0.0,0.125000,0.061728,0.709668,0.299482
261235,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},did they really retire or was it for a short period?,Are there any other interesting aspects about this article?,0.0,0.000000,0.042373,0.685931,0.299449
120791,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},WHen was his second term?,In what year did Yeltsin undergo emergency heart bypass surgery?,0.0,0.000000,0.076923,0.722384,0.299418
...,...,...,...,...,...,...,...,...,...,...,...,...
118627,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},any intresting information?,When did Chuck Berry return to Chess?,0.0,0.000000,0.113636,0.715174,-0.065768
120979,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},Are there any other interesting aspects about this article?,When did Kanye West become general manager of the Memphis Grizzlies?,0.0,0.000000,0.049020,0.624197,-0.066777
264755,T5_amalgam_quac,T5,amalgam,quac,{'defaults': True},Any others?,When did the album come out?,0.0,0.000000,0.147059,0.695957,-0.081780
120381,bart_amalgam_quac,bart,amalgam,quac,{'defaults': True},What is a Portal approach?,Did they release any albums during this time?,0.0,0.000000,0.079365,0.702668,-0.083666


In [17]:
# Find BART predictions with semantic metric score differences

bart_low = bart_df[
    (bart_df["trained_on"] == "amalgam")
    & (bart_df["bertscore-f1"] <= 0.73)  # top 25th percentile bertscore
    & (bart_df["use"] <= 0.3)   # bottom 25th percentile use
    & (bart_df["tested_on"] == "triviaqa")
    # & ((bart_df["tested_on"] == "quac") | (bart_df["tested_on"] == "triviaqa"))
]

bart_low = bart_low.sort_values(by="use", ascending=False)

bart_low

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
135878,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"Which other colour, along with yellow and gree...",What colour/color is the most common form of c...,0.0,0.240000,0.161290,0.728696,0.298450
135311,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"In 1968, ‘The Troubles’ began in which Europea...",In which British county is the town of Clermis...,0.0,0.214286,0.171429,0.705992,0.296745
138294,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"""Which company uses the advertising slogan """"E...",What is the name of the supermarket chain foun...,0.0,0.086957,0.062893,0.716990,0.295899
140890,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"Oona was the last of his four wives, and he di...","""Who was known as """"The Tramp""""?""",0.0,0.173913,0.077720,0.688599,0.295725
139969,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},Which 1989 music video of Madonna attracted cr...,Which song by Madonna reached number one in th...,0.0,0.081633,0.056180,0.706499,0.294621
...,...,...,...,...,...,...,...,...,...,...,...,...
142941,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"Weighing-in at up 48 stone, who wrestled under...",What was the real name of Martin Ruane?,0.0,0.181818,0.074074,0.695134,0.003383
136053,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},Which car manufacturer made models called 'Ens...,What is ISO 26262?,0.0,0.000000,0.044248,0.648023,-0.000665
136925,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"The German 'Dosenwerfen', the French 'Chamboul...",A coconut shy is what type of game?,0.0,0.000000,0.029240,0.687959,-0.012529
137304,bart_amalgam_triviaqa,bart,amalgam,triviaqa,{'defaults': True},"Ted Nevin, who died in 1992, was a leading aut...",Which river forms the border between Europe an...,0.0,0.000000,0.032468,0.673559,-0.017568


Look at context for examples above

In [31]:
prediction_root = "/content/drive/MyDrive/w266 NLP Final Project/Predictions/"
filename = join(prediction_root, "predictions.bart_base_pt_long.amalgam.triviaqa.csv")

csv_df = pd.read_csv(filename)

In [34]:
csv_df = csv_df[csv_df["target"] == "Who's missing: Inky, Pinky, and Sue"]

csv_df

context  \
3942                                                                                                                                                                             The Ghosts , primarily Blinky , Pinky , Inky and Clyde , are the monsters chasing the player in the Pac-Man franchise . \n \n History \n \n The enemies in Pac-Man are known variously as `` ghosts , '' `` goblins , '' and `` monsters '' . \n Despite the seemingly random nature of the enemies , their movements are strictly deterministic , which players have used to their advantage . In an interview , creator Toru Iwatani stated that he had designed each enemy with its own distinct personality in order to keep the game from becoming impossibly difficult or boring to play . More recently , Iwatani described the enemy behaviors in more detail at the 2011 Game Developers Conference . He stated that the red enemy chases Pac-Man , and the pink and blue enemies try to position themselves in front of Pac-Man 's mouth . Although he claimed that the orange enemy 's behavior is random , the game 's code reveals that it actually chases Pac-Man most of the time , but also moves toward the lower-left corner of the maze when it gets too close to Pac-Man . \n \n Cartoons \n \n In the Pac-Man cartoon , with the male Ghost Monsters wearing various styles of hats and partnered with a new ghost named Sue ( who was purple and wore earrings ) , the Ghost Monsters work for Mezmaron who assigns them the job of finding the Power Pellet Forest . Just as Pac-Man could `` chomp '' the Ghost Monsters to devour their `` suits '' after gulping down a power pellet and thus turning the Ghost Monsters blue — with their disembodied `` eyes '' floating back to Mezmaron 's lair — the Ghost Monsters themselves , whenever Pac-Man had n't been able to consume a power pellet for a time , could likewise `` chomp '' Pac-Man to defeat him . The episodes `` Chomp-Out at the O.K . Corral '' and `` The Greatest Show in Pacland '' featured the Ghost Monsters ' 3-year-old cousin named Dinky who was bratty , which greatly annoyed all the male ghost monsters . \n \n In the 2013 TV series Pac-Man and the Ghostly Adventures , the four Ghosts come from the Netherworld . Though they are ruled by Lord Betrayus , they are actually good-natured spirits and often supply Pac-Man with information about Lord Betrayus ' plots , while ensuring Betrayus does n't catch them in the act . It is also suggested that they could be reunited with their bodies and brought back to life , though their 'living ' forms are unknown . There were also some Ghosts that were exclusive to the TV series like Cyclops Ghosts ( a race of heavyset , horned Ghosts with one eye ) , Fire Ghosts ( a race of orange Ghosts who can emit fire from their body ) , Tentacle Ghosts ( a race of 4-eyed purple-black Ghosts who look similar to jellyfish ) , Guardian Ghosts ( a race of large Ghosts who guard the Netherworld ) , and Aqua Ghosts ( a race of light blue Ghosts with fins on their head ) . \n \n Known Ghosts \n \n Below is the description of each Ghost . \n \n Blinky \n \n Blinky is a red ghost who , in the original arcade game , follows behind Pac-man . He is considered the leader of the ghosts . \n \n In the Pac-Man cartoon , Blinky ( voiced by Chuck McCann ) is slow-witted and cowardly with grammar problems . \n \n In Pac-Man and the Ghostly Adventures , Pac-Man and the Ghostly Adventures 2 , & Mappy : The Movie , Blinky ( voiced by Ian James Corlett in the TV series and by Lucien Dodge in the video game ) is the default leader of the Ghost Gang Family and tends to help the winning side . \n \n Blinky receives a speed boost after a number of pac-pellets have been cleared . This mode has been informally referred to as `` Cruise Elroy '' . \n \n Pinky \n \n Pinky is a pink ghost who , in the original arcade game , positions him/herself in front of Pac-man . \n \n In the Pac-Man cartoon , Pinky ( voiced by Chuck McCann ) is depicted as m

### Set Up Probe

In [26]:
class Probe:
    """
    A class used to represent an Awesome Puppies Question Generation Probe
    ...

    Attributes
    ----------
    models : dict
        a dictionary of cached models
    tokenizers : dict
        a dictionary of cached tokenizers

    Methods
    -------
    predict(context, answer, base_model='bart', training_dataset='amalgam',
            num_beams=4, early_stopping=True, no_repeat_ngram_size=3,
            maximum_input_length=1024, maximum_target_length=50)

        returns a prediction string

    retrieve_tokenizer (base_model='bart')

        returns and caches a tokenizer

    retrieve_model(base_model='bart', training_dataset='amalgam')

        returns and caches a model
    """

    project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
    #    project_root = "/home/localadmin/Documents/w266_NLP_Final_Project/"
    model_root = project_root + "Models/"

    def __init__(self):
        self.models = {}
        self.tokenizers = {}

    def predict(
        self,
        context,
        answer,
        base_model="bart",
        training_dataset="amalgam",
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,
        maximum_input_length=1024,
        maximum_target_length=50,
    ):

        tokenizer = self.retrieve_tokenizer(base_model)
        model = self.retrieve_model(base_model, training_dataset)

        if base_model == "bart":
            prompt_string = f"{answer}  {context}"
        elif base_model == "T5":
            prompt_string = f"generate question: answer: {answer} context: {context}"
        else:
            raise ValueError("invalid base model")

        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            max_length=maximum_input_length,
            truncation=True,
            padding=True,
        )
        output_ids = model.generate(
            inputs["input_ids"].cuda(),
            max_length=maximum_target_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )
        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return prediction

    def retrieve_tokenizer(self, base_model="T5"):
        tokenizer = self.tokenizers.get(base_model)
        if tokenizer:
            return tokenizer

        if base_model == "bart":
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        elif base_model == "T5":
            tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
        else:
            raise ValueError("invalid base model")

        self.tokenizers[base_model] = tokenizer
        return tokenizer

    def retrieve_model(self, base_model="bart", training_dataset="amalgam"):
        model_tuple = (base_model, training_dataset)
        model = self.models.get(model_tuple)
        if model:
            return model

        model_dir = f"{self.model_root}{base_model}_base_pt_long.{training_dataset}"

        if base_model == "bart":
            print(f"Loading: {model_dir}")
            model = BartForConditionalGeneration.from_pretrained(model_dir)
        elif base_model == "T5":
            print(f"Loading: {model_dir}")
            model = T5ForConditionalGeneration.from_pretrained(model_dir)
        else:
            raise ValueError("invalid base model")

        model.to(torch.device("cuda:0"))
        self.models[model_tuple] = model
        return model

In [27]:
def panel(
    probe,
    context,
    answers,
    base_models,
    training_datasets,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=3,
    maximum_input_length=1024,
    maximum_target_length=50,
):
    result = [
        {
            "Base Model": base_model,
            "Training Dataset": training_dataset,
            "Answer": answer,
            "Prediction": "".join(
                probe.predict(
                    context,
                    answer,
                    base_model=base_model,
                    training_dataset=training_dataset,
                    num_beams=num_beams,
                    no_repeat_ngram_size=no_repeat_ngram_size,
                    early_stopping=early_stopping,
                    maximum_input_length=maximum_input_length,
                    maximum_target_length=maximum_target_length,
                )
            ),
        }
        for answer in answers
        for base_model in base_models
        for training_dataset in training_datasets
    ]
    replacements = {
        "bart": "BART",
        "amalgam": "Shuffled Blended",
        "nq": "NQ",
        "quac": "QuAC",
        "squad": "SQuAD",
        "triviaqa": "TriviaQA",
    }
    return pd.DataFrame(result).replace(replacements)

Analysis with Probe

In [28]:
p = Probe()

In [ ]:
df = panel(
    p,
    fossie_had_a_little_lamb,
    ["Fossie", "little", "white"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)
df

## Format code to PEP 8 Standards

### Steps

*   Install:

In [ ]:
!pip install black[jupyter]


*   To format your code run:

In [ ]:
!black /content/drive/MyDrive/'Colab Notebooks'/results_text_evaluation.ipynb


*   Don't save your notebook, hit F5 (Command + r) to refresh the page
*   Voila!
*   Now save!